In [23]:
import nest_asyncio
from ib_insync import IB, Index

# Allow nested asyncio loops
nest_asyncio.apply()

# Initialize IB instance
ib = IB()
ib.connect('127.0.0.1', 7496, clientId=1)

print("Connection successful:", ib.isConnected())

# Request historical data for DJIA
# Define the DIA ETF contract
contract = Index('INDU')

# Qualify the contract
qualified_contract = ib.qualifyContracts(contract)
print(qualified_contract)
# Convert to DataFrame


Connection successful: True
[Index(conId=1935181, symbol='INDU', exchange='CME', currency='USD', localSymbol='DJI')]


In [24]:
from datetime import datetime, timedelta
import pandas as pd
# Set the time range
end_date = datetime.now()  # End at current time
start_date = end_date - timedelta(days=365)  # Go back 1 year

# Initialize an empty DataFrame to store results
all_data = []

# Loop through each day to fetch minute data
current_date = end_date
while current_date > start_date:
    print(f"Fetching data for {current_date.date()}...")
    try:
        bars = ib.reqHistoricalData(
            contract,
            endDateTime=current_date.strftime('%Y%m%d %H:%M:%S'),
            durationStr='1 D',  # Fetch 1 day at a time
            barSizeSetting='1 min',  # 1-minute bars
            whatToShow='TRADES',
            useRTH=True,  # Only regular trading hours
            formatDate=1
        )
        # Convert bars to DataFrame and append to all_data
        df = pd.DataFrame([{
            'date': bar.date,
            'open': bar.open,
            'high': bar.high,
            'low': bar.low,
            'close': bar.close,
            'volume': bar.volume
        } for bar in bars])
        all_data.append(df)

    except Exception as e:
        print(f"Error fetching data for {current_date.date()}: {e}")

    # Move to the previous day
    current_date -= timedelta(days=1)

# Combine all daily data into one DataFrame
final_data = pd.concat(all_data, ignore_index=True)

# Save to CSV
final_data.to_csv('minute_data.csv', index=False)
print("Data saved to 'minute_data.csv'")

# Disconnect from IB
ib.disconnect()

Fetching data for 2025-01-25...
Fetching data for 2025-01-24...
Fetching data for 2025-01-23...
Fetching data for 2025-01-22...
Fetching data for 2025-01-21...
Fetching data for 2025-01-20...
Fetching data for 2025-01-19...
Fetching data for 2025-01-18...
Fetching data for 2025-01-17...
Fetching data for 2025-01-16...
Fetching data for 2025-01-15...
Fetching data for 2025-01-14...
Fetching data for 2025-01-13...
Fetching data for 2025-01-12...
Fetching data for 2025-01-11...
Fetching data for 2025-01-10...
Fetching data for 2025-01-09...
Fetching data for 2025-01-08...
Fetching data for 2025-01-07...
Fetching data for 2025-01-06...
Fetching data for 2025-01-05...
Fetching data for 2025-01-04...
Fetching data for 2025-01-03...
Fetching data for 2025-01-02...
Fetching data for 2025-01-01...
Fetching data for 2024-12-31...
Fetching data for 2024-12-30...
Fetching data for 2024-12-29...
Fetching data for 2024-12-28...
Fetching data for 2024-12-27...
Fetching data for 2024-12-26...
Fetching

Error 162, reqId 8: Historical Market Data Service error message:HMDS query returned no data: DJI@CME Trades, contract: Index(conId=1935181, symbol='INDU', exchange='CME', currency='USD', localSymbol='DJI')
Error 162, reqId 19: Historical Market Data Service error message:HMDS query returned no data: DJI@CME Trades, contract: Index(conId=1935181, symbol='INDU', exchange='CME', currency='USD', localSymbol='DJI')
Error 162, reqId 61: Historical Market Data Service error message:HMDS query returned no data: DJI@CME Trades, contract: Index(conId=1935181, symbol='INDU', exchange='CME', currency='USD', localSymbol='DJI')
Error 162, reqId 148: Historical Market Data Service error message:HMDS query returned no data: DJI@CME Trades, contract: Index(conId=1935181, symbol='INDU', exchange='CME', currency='USD', localSymbol='DJI')
Error 162, reqId 208: Historical Market Data Service error message:HMDS query returned no data: DJI@CME Trades, contract: Index(conId=1935181, symbol='INDU', exchange='

In [62]:
# Load the CSV file into a DataFrame
df = pd.read_csv('minute_data.csv')

df = df.drop_duplicates()
# Display the first few rows
print(df.head())

                        date     open     high      low    close  volume
0  2025-01-24 08:30:00-06:00  44533.8  44533.8  44470.6  44516.8     0.0
1  2025-01-24 08:31:00-06:00  44519.6  44544.7  44490.1  44495.6     0.0
2  2025-01-24 08:32:00-06:00  44493.8  44527.0  44490.6  44498.9     0.0
3  2025-01-24 08:33:00-06:00  44499.4  44506.7  44480.1  44487.6     0.0
4  2025-01-24 08:34:00-06:00  44489.4  44509.4  44483.5  44483.6     0.0


In [63]:
# Ensure 'datetime' column is in datetime format
df['date'] = pd.to_datetime(df['date'],  utc=True)
df.head()

,date,open,high,low,close,volume
0,2025-01-24 14:30:00+00:00,44533.8,44533.8,44470.6,44516.8,0.0
1,2025-01-24 14:31:00+00:00,44519.6,44544.7,44490.1,44495.6,0.0
2,2025-01-24 14:32:00+00:00,44493.8,44527.0,44490.6,44498.9,0.0
3,2025-01-24 14:33:00+00:00,44499.4,44506.7,44480.1,44487.6,0.0
4,2025-01-24 14:34:00+00:00,44489.4,44509.4,44483.5,44483.6,0.0


In [64]:
# Group by day (extract date from the datetime column)
groups = df.groupby(df['date'].dt.date)

# Process each group
for day, group in groups:
    print(f"Data for {day}:")
    print(group)
    print()

Data for 2024-01-26:
                            date     open     high      low    close  volume
143370 2024-01-26 14:30:00+00:00  38006.7  38044.7  37997.7  38017.4     0.0
143371 2024-01-26 14:31:00+00:00  38019.4  38070.2  38014.2  38070.1     0.0
143372 2024-01-26 14:32:00+00:00  38073.7  38106.9  38068.3  38096.6     0.0
143373 2024-01-26 14:33:00+00:00  38098.0  38098.0  38074.9  38076.5     0.0
143374 2024-01-26 14:34:00+00:00  38076.5  38089.8  38067.1  38068.8     0.0
...                          ...      ...      ...      ...      ...     ...
143770 2024-01-26 21:10:00+00:00  38109.4  38109.4  38109.4  38109.4     0.0
143771 2024-01-26 21:11:00+00:00  38109.4  38109.4  38109.4  38109.4     0.0
143772 2024-01-26 21:12:00+00:00  38109.4  38109.4  38109.4  38109.4     0.0
143773 2024-01-26 21:13:00+00:00  38109.4  38109.4  38109.4  38109.4     0.0
143774 2024-01-26 21:14:00+00:00  38109.4  38109.5  38109.4  38109.4     0.0

[405 rows x 6 columns]

Data for 2024-01-29:
         

In [65]:
group_sizes = groups.size()
average_length = group_sizes.mean()  # Average length
max_length = group_sizes.max()       # Maximum length
min_length = group_sizes.min()       # Minimum length

# Display results
print("Group Sizes:")
print(group_sizes)
print("\nMetrics:")
print(f"Average Length: {average_length}")
print(f"Maximum Length: {max_length}")
print(f"Minimum Length: {min_length}")

Group Sizes:
date
2024-01-26    405
2024-01-29    405
2024-01-30    405
2024-01-31    405
2024-02-01    405
             ... 
2025-01-17    405
2025-01-21    405
2025-01-22    405
2025-01-23    405
2025-01-24    405
Length: 250, dtype: int64

Metrics:
Average Length: 405.0
Maximum Length: 405
Minimum Length: 405


In [66]:

one_day_data = groups.get_group(pd.to_datetime('2025-01-17').date())

In [67]:
flattened_one_day = one_day_data.drop(['date', 'volume'], axis=1).stack()

In [68]:
trend_transform = []

for date, candle in one_day_data.iterrows():
    if candle['open'] > candle['close']:
        trend_transform += [candle['open'], candle['high'], candle['low'], candle['close']]
    else:
        trend_transform += [candle['open'], candle['low'], candle['high'], candle['close']]


In [71]:
import plotly.express as px

fig = px.line(trend_transform)
fig.show()

In [73]:
import plotly.express as px

fig = px.line(x=one_day_data['date'], y=one_day_data['close'])
fig.show()